In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy.stats import pearsonr
import numpy as np

csv_fem_lab = "../Resources/labor_force_female.csv"
csv_male_lab = "../Resources/labor_male.csv"
csv_total_lab = "../Resources/labor_total.csv"
csv_lab_perc_fem = "../Resources/labor_perc_fem.csv"
csv_mortality_5 = "../Resources/mortality_under5.csv"


# Labor force participation rate(% of population ages 15-64); among women, men, and total
fem_lab = pd.read_csv(csv_fem_lab)
male_lab = pd.read_csv(csv_male_lab)
total_lab = pd.read_csv(csv_total_lab)
mortality_under_5 = pd.read_csv(csv_mortality_5)

# % of labor force that are women (will reflect any skew in gender proportions in a given country)
lab_perc_fem = pd.read_csv(csv_lab_perc_fem)
mortality_under_5.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
1960              float64
1961              float64
1962              float64
1963              float64
1964              float64
1965              float64
1966              float64
1967              float64
1968              float64
1969              float64
1970              float64
1971              float64
1972              float64
1973              float64
1974              float64
1975              float64
1976              float64
1977              float64
1978              float64
1979              float64
1980              float64
1981              float64
1982              float64
1983              float64
1984              float64
1985              float64
                   ...   
1988              float64
1989              float64
1990              float64
1991              float64
1992              float64
1993              float64
1994              float64
1995        

In [2]:
# Function to clean up data sets and use only country-level data, no regions or WB categories
country_list = ['AFG', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'AIA', 'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLX', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BAT', 'BRA', 'IOT', 'VGB', 'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CYM', 'CAF', 'TCD', 'CHL', 'CHN', 'CXR', 'CCK', 'COL', 'COM', 'ZAR', 'COG', 'COK', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'CSK', 'DNK', 'DJI', 'DMA', 'DOM', 'TMP', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'ETF', 'EUN', 'FRO', 'FLK', 'FJI', 'FIN', 'PCZ', 'ZW1', 'TAN', 'VDR', 'SVR', 'ZPM', 'ATF', 'FRA', 'FRE', 'GUF', 'PYF', 'GAB', 'GMB', 'GAZ', 'GEO', 'DDR', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'VAT', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JTN', 'JOR', 'KAZ', 'KEN', 'KIR', 'PRK', 'KOR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LIE', 'LTU', 'LUX', 'MAC', 'MKD', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL', 'MTQ', 'MRT', 'MUS', 'MEX', 'FSM', 'MID', 'MDA', 'MCO', 'MNG', 'MSR', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD', 'ANT', 'NZE', 'NCL', 'NZL', 'NIC', 'NER', 'NGA', 'NIU', 'NFK', 'MNP', 'NOR', 'OMN', 'PCE', 'PAK', 'PLW', 'PAN', 'PNG', 'PRY', 'PMY', 'PER', 'PHL', 'PCN', 'POL', 'PRT', 'PRI', 'QAT', 'REU', 'ROM', 'RUS', 'RWA', 'RYU', 'SBH', 'SHN', 'KN1', 'SPM', 'WSM', 'SMR', 'STP', 'SWK', 'SAU', 'SEN', 'SYC', 'SLE', 'SIK', 'SGP', 'SVK', 'SVN', 'SLB', 'SOM', 'ZAF', 'SVU', 'ESP', 'SPE', 'LKA', 'KNA', 'LCA', 'VCT', 'SDN', 'SUR', 'SJM', 'SWZ', 'SWE', 'CHE', 'SYR', 'TWN', 'TJK', 'TZA', 'THA', 'TGO', 'TKL', 'TON', 'TTO', 'TUN', 'TUR', 'TKM', 'TCA', 'TUV', 'UGA', 'UKR', 'ARE', 'GBR', 'USA', 'UNS', 'URY', 'USP', 'UZB', 'VUT', 'VEN', 'VNM', 'VIR', 'WAK', 'WLF', 'ESH', 'WLD', 'YDR', 'YEM', 'SER', 'YUG', 'ZMB', 'ZWE']

def clean_df(df):
    df.index = df['Country Code']
    df = df[df['Country Code'].isin(country_list)]
    df = df.drop(columns=['Indicator Code', 'Country Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'])
#     df = df.dropna(how='any', thresh=3)
    return df

fem_lab = clean_df(fem_lab)
male_lab = clean_df(male_lab)
total_lab = clean_df(total_lab)
lab_perc_fem = clean_df(lab_perc_fem)
mortality_under_5 = clean_df(mortality_under_5)

In [3]:
mortality_under_5.to_csv('mortality_under5.csv')

In [4]:
lab_ratio = fem_lab[['1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017']] / male_lab[['1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017']]
# lab_ratio_dif = (fem_lab['2017'] - fem_lab['1990']) / fem_lab['1990']
lab_ratio.head()

1990      1991      1992      1993      1994      1995  \
Country Code                                                               
ABW                NaN       NaN       NaN       NaN       NaN       NaN   
AFG           0.170580  0.171387  0.172137  0.172345  0.172433  0.173226   
AGO           0.921183  0.919662  0.918335  0.914296  0.915948  0.918874   
ALB           0.720141  0.745257  0.749689  0.740724  0.734181  0.726140   
AND                NaN       NaN       NaN       NaN       NaN       NaN   

                  1996      1997      1998      1999    ...         2008  \
Country Code                                            ...                
ABW                NaN       NaN       NaN       NaN    ...          NaN   
AFG           0.171362  0.169687  0.168608  0.168045    ...     0.171435   
AGO           0.919928  0.921029  0.922410  0.923605    ...     0.935467   
ALB           0.718269  0.722567  0.715310  0.706948    ...     0.722840   
AND                NaN       NaN       NaN       NaN    ...          NaN   

                  2009      2010      2011      2012      2013      2014  \
Country Code                                                               
ABW                NaN       NaN       NaN       NaN       NaN       NaN   
AFG           0.169559  0.169958  0.176328  0.184791  0.194735  0.205303   
AGO           0.935406  0.935750  0.935299  0.935699  0.936869  0.938037   
ALB           0.699076  0.725470  0.731304  0.738176  0.712735  0.701121   
AND                NaN       NaN       NaN       NaN       NaN       NaN   

                  2015      2016      2017  
Country Code                                
ABW                NaN       NaN       NaN  
AFG           0.215845  0.223935  0.224571  
AGO           0.939203  0.939573  0.940468  
ALB           0.731665  0.729236  0.727965  
AND                NaN       NaN       NaN  

[5 rows x 28 columns]

In [5]:
lab_ratio.to_csv('lab_ratio.csv')
# lab_ratio_dif.to_csv('lab_ratio_dif.csv')

In [6]:
# generating correlation and test of significance between two dataframes

# dropping all columns aside from years
df1 = male_lab[['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017']]
df2 = fem_lab[['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017']]

coeffmat = np.zeros((df1.shape[1], 2))

for i in range(df1.shape[1]):
    corrtest = pearsonr(df1[df1.columns[i]], df2[df2.columns[i]])  
    coeffmat[i,0] = corrtest[0]
    coeffmat[i,1] = corrtest[1]

dfcorrelation = pd.DataFrame(coeffmat, columns={'coeff', 'pval'}, index=df1.columns)
# dfpvals = pd.DataFrame(pvalmat, columns=df2.columns, index=df1.columns)

In [7]:
male_lab_mean = male_lab.mean()
male_lab_min = male_lab.min()
male_lab_max = male_lab.max()
fem_lab_mean = fem_lab.mean()
fem_lab_min = fem_lab.min()
fem_lab_max = fem_lab.max()
dfcorrelation['mean total labor participation'] = total_lab_mean
dfcorrelation['min labor participation'] = total_lab_min
dfcorrelation['max labor participation'] = total_lab_max
dfcorrelation['mean female labor participation'] = fem_lab_mean
dfcorrelation['min female labor participation'] = fem_lab_min
dfcorrelation['max female labor participation'] = fem_lab_max
dfcorrelation['predicted min female labor participation'] = dfcorrelation['mean female labor participation'] - ((dfcorrelation['mean total labor participation'] 
                                             - dfcorrelation['min labor participation']) * dfcorrelation['coeff'])
dfcorrelation['predicted max female labor participation'] = dfcorrelation['mean female labor participation'] + ((dfcorrelation['max labor participation']
                                            - dfcorrelation['mean total labor participation']) * dfcorrelation['coeff'])
dfcorrelation = dfcorrelation.reindex(['min labor participation', 'mean total labor participation', 
                                       'max labor participation', 'min female labor participation', 'predicted min female labor participation', 'mean female labor participation', 
                                       'max female labor participation', 'predicted max female labor participation', 'coeff', 'pval'], axis=1)

NameError: name 'total_lab_mean' is not defined

In [ ]:
#To correlate all columns with each other, not just the corresponding columns of each dataframe:

# coeffmat = np.zeros((df1.shape[1], df2.shape[1]))
# pvalmat = np.zeros((df1.shape[1], df2.shape[1]))

# for i in range(df1.shape[1]):    
#     for j in range(df2.shape[1]):        
#         corrtest = pearsonr(df1[df1.columns[i]], df2[df2.columns[j]])  

#         coeffmat[i,j] = corrtest[0]
#         pvalmat[i,j] = corrtest[1]

# dfcoeff = pd.DataFrame(coeffmat, columns=df2.columns, index=df1.columns)
# print(dfcoeff)

# dfpvals = pd.DataFrame(pvalmat, columns=df2.columns, index=df1.columns)
# print(dfpvals)

In [ ]:
dfcorrelation.to_csv('dfcorrelation.csv')
dfcorrelation